In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np
import pandas as pd
import ast
import math

In [20]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_size, input_size)
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input_seq, hidden_state):
        embedding = self.embedding(input_seq)
        output, hidden_state = self.rnn(embedding, hidden_state)
        output = self.decoder(output)
        return output, (hidden_state[0].detach(), hidden_state[1].detach())
    

In [21]:
data = pd.read_csv('./data/eqs.csv')

In [22]:
y = data["eqs"]

In [23]:
X = data.drop('eqs', axis = 1)

In [43]:
def int_to_binary(num, width):
    if num >= 0:
        binary = bin(num)[2:].zfill(width)
        return int("0" + binary) 
    else:
        binary = bin(abs(num))[2:].zfill(width)
        return int("1" + binary)

def binary_to_int(str, width):
    sign = str[0]
    num = str[1:]
    num = int(num, 2)

    if sign == "0":
        return num
    else:
        return num * - 1

In [46]:
eq = y[0]

In [47]:
[int_to_binary(x, 6) for x in X.head(1).to_numpy()[0]]

[0, 11000, 11100, 100000, 100100, 101000, 101100, 110000, 110100, 111000]

In [44]:
seq = [torch.Tensor(int_to_binary(x, 6)) for x in X.head(1).to_numpy()[0]]

In [45]:
seq

[tensor([]),
 tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 tensor([0.0000e+00, 0.0000e+00, 1.0377e-31,  ..., 1.8315e+25, 4.7415e+16,
         1.3497e+28]),
 tensor([0.0000e+00, 0.0000e+00, 1.0377e-31,  ..., 6.1897e-04, 2.5204e-12,
         1.8376e+37]),
 tensor([0.0000e+00, 0.0000e+00, 1.0377e-31,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]),
 tensor([0.0000e+00, 0.0000e+00, 1.0377e-31,  ..., 2.9388e-39, 2.9388e-39,
         7.0714e-39]),
 tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 tensor([0., 0., 0.,  ..., 0., 0., 0.])]